In [ ]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import numpy as np
import dataclasses
import matplotlib.pyplot as plt
import seaborn as sns
import genjax

sns.set_theme(style="white")

# Pretty printing.
console = genjax.pretty(width=80)

# Reproducibility.
key = jax.random.PRNGKey(314159)

The generative function interface exposes functionality which allows usage of generative functions for differentiable programming. These interfaces are designed to work seamlessly with `jax.grad` - allowing (even higher-order) gradient computations which are useful for inference algorithms which require gradients and gradient estimators. In this notebook, we'll describe some of these interfaces - as well as their (current, but not forever) limitations. We'll walk through an implementation of MAP estimation, as well as the Metropolis-adjusted Langevin algorithm (MALA) using these interfaces.

## Gradient interfaces

Because JAX features best-in-class support for higher-order AD, GenJAX exposes interfaces that compose natively with JAX's interfaces for gradients. The primary interface method which provides `jax.grad`-compatible functions from generative functions is an interface called `unzip`. 

`unzip` allows a user to provide a key, and a fixed choice map - and it returns a new key and two closures:

* The first closure is a "score" closure which accepts a choice map as the first argument, and arguments which match the non-`PRNGKey` signature types of the generative function. The score closure returns the exact joint score of the generative function. It computes the exact joint score using an interface called `assess`.^[Caveat: `assess` is not required to return the _exact_ joint score, only an estimate. However, if `jax.grad` is used on estimates - the resulting thing is not a correct gradient estimator. See the important callout below!]
* The second closure is a "retval" closure which accepts a choice map as the first argument, and arguments which match the non-`PRNGKey` signature types of the generative function. The retval closure executes the generative function constrained using the union of the fixed choice map, and the user provided choice map, and returns the return value of the execution. Here, the return value is also provided by invoking the `assess` interface.

So really, `unzip` is syntactic sugar over another interface called `assess`.

### `assess` for exact density evaluation

`assess` is a generative function interface method which computes log joint density estimates from generative functions. `assess` requires that a user provide a choice map _which completely fills all choices encountered during execution_. Otherwise, it errors.^[And these errors are thrown at JAX trace time, so you'll get an exception before runtime.]

If a generative function also draws from untraced randomness - `assess` computes an estimate whose expectation over the distribution of untraced randomness gives the correct log joint density. 

::: {.callout-important}

## Correctness of gradient estimators

When used on generative functions which include untraced randomness, naively applying `jax.grad` to the closures returned by interfaces described in this notebook **do not compute** gradient estimators which are unbiased with respect to the true gradients.

Short: don't use these with untraced randomness. We're working on alternatives.

:::

## A running example

Let's consider the following model, which we'll cover in different variations.

In [ ]:
@genjax.gen
def model(key):
    key, x_mu = genjax.trace("x_mu", genjax.Uniform)(key, -3.0, 3.0)
    key, a = genjax.trace("a", genjax.Uniform)(key, -3.0, 3.0)
    key, b = genjax.trace("b", genjax.Uniform)(key, -3.0, 3.0)
    key, x = genjax.trace("x", genjax.Normal)(key, x_mu, 1.0)
    return genjax.trace("y", genjax.Normal)(key, a * x + b, 1.0)

And, most importantly, visualizations.

In [ ]:
def viz(ax, x, y, **kwargs):
    sns.scatterplot(x=x, y=y, ax=ax, **kwargs)


f, axes = plt.subplots(3, 3, figsize=(9, 9), sharex=True, sharey=True, dpi=280)
jitted = jax.jit(jax.vmap(model.simulate, in_axes=(0, None)))
for ax in axes.flatten():
    key, *sub_keys = jax.random.split(key, 30 + 1)
    sub_keys = jnp.array(sub_keys)
    _, tr = jitted(sub_keys, ())
    x = tr["x"]
    y = tr["y"]
    viz(ax, x, y, marker=".")

plt.show()

A nice diffuse prior over points.

## MAP estimation

A good first step into the gradient waters is gradient-based maximum a posteriori probability (MAP) estimation. Here's how we write this in GenJAX:

In [ ]:
@dataclasses.dataclass
class MapUpdate(genjax.Pytree):
    selection: genjax.Selection
    tau: genjax.Float

    def flatten(self):
        return (self.tau,), (self.selection,)

    def apply(self, key, trace):
        args = trace.get_args()
        gen_fn = trace.get_gen_fn()
        key, forward_gradient_trie = gen_fn.choice_grad(key, trace, selection)
        forward_values, _ = self.selection.filter(trace)
        forward_values = forward_values.strip()
        forward_values = jtu.tree_map(
            lambda v1, v2: v1 + self.tau * v2,
            forward_values,
            forward_gradient_trie,
        )
        key, (_, _, new_trace, _) = gen_fn.update(
            key, trace, forward_values, args
        )
        return key, (new_trace, True)

    def __call__(self, key, trace):
        return self.apply(key, trace)


map_update = MapUpdate

Simple, concise - works with any generative function whose addresses in `MapUpdate.selection` are modelled as continuous random variables.

From the `Pytree` interface, any instance of `MapUpdate` has a static `selection`, and a `tau` (which determines the gradient step size) which can be dynamic.

In inference code, we'd just construct `MapUpdate` before calling it.

In [ ]:
update = map_update(genjax.select(["x_mu", "a", "b"]), 0.3)
update

## Exposing learnable modules with `TrainCombinator`

For learning and variational inference, learnable parameters of model families are an important feature. In GenJAX, we expose a lightweight generative function wrapper around generative functions which accept `learnable_params` as a last argument - this wrapper is called `TrainCombinator`.

In [ ]:
params = {"x_mu": 0.0, "a": 0.3, "b": 0.4}


@genjax.gen(genjax.TrainCombinator, params=params)
def model(key, params):
    x_mu = params["x_mu"]
    a = params["a"]
    b = params["b"]
    key, x = genjax.trace("x", genjax.Normal)(key, x_mu, 1.0)
    return genjax.trace("y", genjax.Normal)(key, a * x + b, 1.0)


model

`TrainCombinator` is a module-like abstraction which closes over the parameter store passed in and initialized by the constructor. When we call `TrainCombinator`, we don't need to provide the `params` argument - it's handled automatically.

In [ ]:
key, tr = model.simulate(key, ())
tr

`TrainCombinator` exposes a convenient interface to a specialized scoring function which accepts `params` evaluation points, and returns the model logpdf.^[Note: because this interface method returns a **function**, we cannot JIT it. However, if we use it to produce a closure and then use that closure inside of code which we JIT, it's fine. As long as an object of **function** type doesn't try to escape across the JAX API boundary.]

In [ ]:
key, logpdf = model.score_params(key, tr, params)
logpdf

This interface supports batching out of the box.

In [ ]:
key, *sub_keys = jax.random.split(key, 100 + 1)
sub_keys = jnp.array(sub_keys)
sub_keys, tr = jax.vmap(model.simulate)(sub_keys, ())
_, logpdf = jax.vmap(model.score_params, in_axes=(0, 0, None))(
    sub_keys, tr, params
)
logpdf

We make extensive use of batch evaluation in variational inference. For now, let's consider maximum likelihood learning and see the other `TrainCombinator` interfaces.

## Automatic differentiation variational inference

In this section, we'll show how we can use the gradient interfaces to implement [Automatic differentiation variational inference](https://arxiv.org/abs/1603.00788).